# Create the Adaptive Capacity Matrix

The objective of this notebook is to create the final output for the adaptive capacity survey. The output is a matrix where the columns are the different indicators of adaptive capacity and the rows are fishers (respodents). Demographic information will also be included.

**Assets**<br>
AS1: Material goods <br>
AS2: Fishing assets <br>
AS3: Access to savings, credit, and loans<br>
**Organization** <br>
O1: Pariticipation in community groups <br>
O2: Membership in CCP <br>
O3: Participation in CCP <br>
O4: Capacity to act collectively <br>
**Learning** <br>
L1: Ability to recognize change <br>
L2: Ability to recognize causility <br>
L3: Ability to recognize SG ES <br>
L4: Knowledge of seagrass management <br>
L5: Formal education <br>
L6: Local knowledge <br>
**Flexibility**<br>
F1: Capacity to modify fishing practice <br>
F2: Capacity to leave fishing <br>
**Agency**<br>
AG1: Ability to respond to change <br>
AG2: Decision making power<br>

In [265]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from zipfile import ZipFile
import seaborn as sns
import os
import math

In [266]:
df = pd.read_csv('/home/rthoms/ocean_dep/sur_C/adaptive_cap_clean.csv')
df_msl = pd.read_csv('/home/rthoms/ocean_dep/sur_C/msl_scores.csv')
df_fishing_assets = pd.read_csv('/home/rthoms/ocean_dep/sur_C/fishing_asset_scores.csv')
df.shape

(99, 72)

In [267]:
# drop duplicates from the cleaned table (if multiple gears were recorded, each has an entry)
df= df.drop_duplicates(subset=['id'], keep='first')
df.shape

(96, 72)

In [268]:
df = pd.merge(df,df_msl[['id','msl']], how = 'left', on= 'id')
df.shape

(96, 73)

In [269]:
df = pd.merge(df,df_fishing_assets[['id','fish_asset_score']], how = 'left', on= 'id')
df.shape

(96, 74)

In [270]:
df.columns

Index(['Unnamed: 0', 'id', 'date', 'ccp', 'gender', 'age', 'birth_place',
       'mar_status', 'ed_level', 'house_head', 'glean', 'electricity',
       'fridge_freezer', 'fridge_freezer_num', 'water', 'matress',
       'matress_num', 'mobile_phone', 'mobile_phone_num', 'smart_phone',
       'smart_phone_num', 'fan', 'fan_num', 'tv', 'tv_num', 'radio',
       'radio_num', 'stove', 'stove_num', 'transport', 'transport_num',
       'floor', 'floor_other', 'wall', 'wall_other', 'roof', 'roof_other',
       'access_credit_loans', 'community_participation', 'ccp_member',
       'ccp_participation', 'decision_making_manag', 'decision_making_fish',
       'change_seagrass', 'direction_change_seagrass',
       'causes_change_seagrass', 'change_fish', 'direction_change_resources',
       'causes_change_resources', 'link_seagrass_resources',
       'capacity_change_practice', 'capacity_change_livelihood',
       'capacity_organize', 'local_knowledge', 'agency_change_seagrass',
       'agency_chan

In [271]:
# function to pull item from dict
def set_scale(dic,indicator):
    scale=dic[indicator]
    return scale

Indexed value(Xi) = (Actual value - minimum value) / (Maximum value - minimum value) <br>
United Nations Development Programme’s protocol for standardizing indicators for the life expectancy index

In [272]:
def index(val, min, max):
    index = (val-min)/(max-min)
    return index

## Assets
### AS1 
Material Style of Life

In [273]:
df['AS1_index']=df.apply(lambda row: index(row['msl'],df.msl.min(),df.msl.max()), axis=1 )
df[['pract_cat','msl','AS1_index']]

,pract_cat,msl,AS1_index
0,glean,0.260379,0.712085
1,glean,-0.544116,0.260888
2,fish,-0.548678,0.258330
3,glean,-0.571298,0.245643
4,shrimp_net,0.054327,0.596522
...,...,...,...
91,shrimp_net,0.042069,0.589647
92,shrimp_net,0.143856,0.646734
93,shrimp_net,0.206149,0.681670
94,glean,-0.159420,0.476643


### AS2
Fishing assets

In [274]:
df['AS2_index']=df.apply(lambda row: index(row['fish_asset_score'],df.fish_asset_score.min(),df.fish_asset_score.max()), axis=1 )
df[['id','pract_cat','fish_asset_score','AS2_index']]

,id,pract_cat,fish_asset_score,AS2_index
0,05032023JOSC01,glean,-0.150443,0.045024
1,05032023JOSC02,glean,-0.150443,0.045024
2,05032023JOSC03,fish,-0.150443,0.045024
3,05032023JOSC04,glean,-0.150443,0.045024
4,05032023JOSC05,shrimp_net,-0.106736,0.074802
...,...,...,...,...
91,19022023ILHC03,shrimp_net,-0.106736,0.074802
92,19022023ILHC04,shrimp_net,-0.106736,0.074802
93,19022023ILHc06,shrimp_net,-0.106736,0.074802
94,19022023ILHC05,glean,-0.150443,0.045024


### AS3
Access to savings, credit, and loans

In [275]:
print(df['access_credit_loans'].unique())

['none' 'rotating_savings' 'NR' 'bank']


In [276]:
# create a dictionary to replace descriptive scale values with interger scale values
credit_loan_dict = {'bank':2,'rotating_savings':1,'none':0,'NR': None}

# apply function
df['AS3_scalar']=df.apply(lambda row: set_scale(credit_loan_dict, row['access_credit_loans']), axis=1 )
df[['pract_cat','access_credit_loans','AS3_scalar']]


,pract_cat,access_credit_loans,AS3_scalar
0,glean,none,0.0
1,glean,rotating_savings,1.0
2,fish,none,0.0
3,glean,none,0.0
4,shrimp_net,none,0.0
...,...,...,...
91,shrimp_net,none,0.0
92,shrimp_net,rotating_savings,1.0
93,shrimp_net,none,0.0
94,glean,none,0.0


In [277]:
df['AS3_index']=df.apply(lambda row: index(row['AS3_scalar'],0,2), axis=1 )
df[['pract_cat','access_credit_loans','AS3_scalar','AS3_index']]

,pract_cat,access_credit_loans,AS3_scalar,AS3_index
0,glean,none,0.0,0.0
1,glean,rotating_savings,1.0,0.5
2,fish,none,0.0,0.0
3,glean,none,0.0,0.0
4,shrimp_net,none,0.0,0.0
...,...,...,...,...
91,shrimp_net,none,0.0,0.0
92,shrimp_net,rotating_savings,1.0,0.5
93,shrimp_net,none,0.0,0.0
94,glean,none,0.0,0.0


## Organization 
### O1 
Participation in community groups

In [278]:
print(df['community_participation'].unique())

['yes' 'no']


In [279]:
# create a dictionary to replace descriptive scale values with interger scale values
y_n_dic = {'yes':1,'no':0,'NR': None, 'not recorded': None}

# apply function
df['O1_scalar']=df.apply(lambda row: set_scale(y_n_dic, row['community_participation']), axis=1 )
df[['pract_cat','community_participation','O1_scalar']]

,pract_cat,community_participation,O1_scalar
0,glean,yes,1
1,glean,yes,1
2,fish,no,0
3,glean,yes,1
4,shrimp_net,no,0
...,...,...,...
91,shrimp_net,no,0
92,shrimp_net,yes,1
93,shrimp_net,no,0
94,glean,no,0


In [280]:
df['O1_index']=df['O1_scalar']

### O2 
Membership in CCP

In [281]:
print(df['ccp_member'].unique())

['non-member' 'member']


In [282]:
# create a dictionary to replace descriptive scale values with interger scale values
member_dic = {'member':1,'non-member':0,'NR': None}

# apply function
df['O2_scalar']=df.apply(lambda row: set_scale(member_dic, row['ccp_member']), axis=1 )
df[['pract_cat','ccp_member','O2_scalar']]

,pract_cat,ccp_member,O2_scalar
0,glean,non-member,0
1,glean,member,1
2,fish,non-member,0
3,glean,non-member,0
4,shrimp_net,non-member,0
...,...,...,...
91,shrimp_net,non-member,0
92,shrimp_net,member,1
93,shrimp_net,non-member,0
94,glean,non-member,0


In [283]:
df['O2_index']=df['O2_scalar']

### O3 
Participation in CCP

In [284]:
print(df['ccp_participation'].unique())

['paticipates regularly' 'does not\nparticipate' 'not recorded']


In [285]:
# create a dictionary to replace descriptive scale values with interger scale values
participation_dic = {'paticipates regularly':1,'does not\nparticipate':0,'not recorded': None}

# apply function
df['O3_scalar']=df.apply(lambda row: set_scale(participation_dic, row['ccp_participation']), axis=1 )
df[['pract_cat','ccp_participation','O3_scalar']]

,pract_cat,ccp_participation,O3_scalar
0,glean,paticipates regularly,1.0
1,glean,paticipates regularly,1.0
2,fish,paticipates regularly,1.0
3,glean,paticipates regularly,1.0
4,shrimp_net,does not\nparticipate,0.0
...,...,...,...
91,shrimp_net,does not\nparticipate,0.0
92,shrimp_net,paticipates regularly,1.0
93,shrimp_net,does not\nparticipate,0.0
94,glean,does not\nparticipate,0.0


In [286]:
df['O3_index']=df['O3_scalar']

### O4 
Capacity to organize

In [287]:
print(df['capacity_organize'].unique())

['agree strongly' 'disagree strongly']


In [288]:
# create a dictionary to replace descriptive scale values with interger scale values
likert_dic = {'agree strongly':2, 'agree slightly':1, 'neutral':0, 'disagree slightly': -1, 'disagree strongly':-2,'NR': None}

# apply function
df['O4_scalar']=df.apply(lambda row: set_scale(likert_dic, row['capacity_organize']), axis=1 )
df[['pract_cat','capacity_organize','O4_scalar']]

,pract_cat,capacity_organize,O4_scalar
0,glean,agree strongly,2
1,glean,agree strongly,2
2,fish,agree strongly,2
3,glean,agree strongly,2
4,shrimp_net,agree strongly,2
...,...,...,...
91,shrimp_net,agree strongly,2
92,shrimp_net,agree strongly,2
93,shrimp_net,agree strongly,2
94,glean,agree strongly,2


In [289]:
df['O4_index']=df.apply(lambda row: index(row['O4_scalar'],-2,2), axis=1 )
df[['pract_cat','capacity_organize','O4_scalar','O4_index']]

,pract_cat,capacity_organize,O4_scalar,O4_index
0,glean,agree strongly,2,1.0
1,glean,agree strongly,2,1.0
2,fish,agree strongly,2,1.0
3,glean,agree strongly,2,1.0
4,shrimp_net,agree strongly,2,1.0
...,...,...,...,...
91,shrimp_net,agree strongly,2,1.0
92,shrimp_net,agree strongly,2,1.0
93,shrimp_net,agree strongly,2,1.0
94,glean,agree strongly,2,1.0


## Learning
### L1
Ability to recognize change

In [290]:
print(df['direction_change_seagrass'].unique())

['less' 'no change' 'significantly less' 'more' 'not recorded' 'not sure'
 'significantly more']


In [291]:
# create a dictionary to replace descriptive scale values with interger scale values
change_dic = {'significantly less':1,'less':1,'no change':0,'not sure':0,'more':0,'significantly more': 0, 'not recorded': None}

# apply function
df['L1_scalar']=df.apply(lambda row: set_scale(change_dic, row['direction_change_seagrass']), axis=1 )
df[['pract_cat','direction_change_seagrass','L1_scalar']]

,pract_cat,direction_change_seagrass,L1_scalar
0,glean,less,1.0
1,glean,no change,0.0
2,fish,significantly less,1.0
3,glean,more,0.0
4,shrimp_net,no change,0.0
...,...,...,...
91,shrimp_net,significantly less,1.0
92,shrimp_net,significantly less,1.0
93,shrimp_net,less,1.0
94,glean,less,1.0


In [292]:
df['L1_index']=df['L1_scalar']

### L2
Ability to recognize causility

In [293]:
print(df['causes_change_seagrass'].unique())

["['climate_change']" "['no_decrease_perceived']" "['destructive_gear']"
 "['destructive_gear', 'climate_change']" "['unsure']" "['NR']"
 "['fishing_pressure', 'destructive_gear']" "['fishing_pressure']"
 "['fishing_pressure', 'destructive_gear', 'climate_change']"
 "['destructive_gear', 'other']" "['other']"]


In [294]:
def recognize(response):
    if 'unsure' in response:
        return 0
    elif 'no_decrease_perceived' in response:
        return None
    elif 'NR' in response: 
        return None
    else:
        return 1

In [295]:
df['L2_scalar']=df.apply(lambda row: recognize(row['causes_change_seagrass']), axis=1 )
df[['pract_cat','causes_change_seagrass','L2_scalar']]

,pract_cat,causes_change_seagrass,L2_scalar
0,glean,['climate_change'],1.0
1,glean,['no_decrease_perceived'],NaN
2,fish,['destructive_gear'],1.0
3,glean,['no_decrease_perceived'],NaN
4,shrimp_net,['no_decrease_perceived'],NaN
...,...,...,...
91,shrimp_net,['climate_change'],1.0
92,shrimp_net,['destructive_gear'],1.0
93,shrimp_net,['other'],1.0
94,glean,['unsure'],0.0


In [296]:
df['L2_index']=df['L2_scalar']

### L3 
Ability to recognize SG ES

In [297]:
print(df['link_seagrass_resources'].unique())

['unsure' 'not associated' 'associated' 'not recorded']


In [298]:
# create a dictionary to replace descriptive scale values with interger scale values
link_dic = {'associated':1,'not associated':0,'unsure':0,'not recorded': None}

# apply function
df['L3_scalar']=df.apply(lambda row: set_scale(link_dic, row['link_seagrass_resources']), axis=1 )
df[['pract_cat','link_seagrass_resources','L3_scalar']]

,pract_cat,link_seagrass_resources,L3_scalar
0,glean,unsure,0.0
1,glean,unsure,0.0
2,fish,not associated,0.0
3,glean,not associated,0.0
4,shrimp_net,associated,1.0
...,...,...,...
91,shrimp_net,not recorded,NaN
92,shrimp_net,not recorded,NaN
93,shrimp_net,not recorded,NaN
94,glean,not recorded,NaN


In [299]:
df['L3_index']=df['L3_scalar']

### L4
Knowledge of seagrass management

In [300]:
print(df['human_mang_seagrass'].unique())

['no' 'yes' 'not recorded']


In [301]:
# apply function
df['L4_scalar']=df.apply(lambda row: set_scale(y_n_dic, row['human_mang_seagrass']), axis=1 )
df[['pract_cat','human_mang_seagrass','L4_scalar']]

,pract_cat,human_mang_seagrass,L4_scalar
0,glean,no,0.0
1,glean,no,0.0
2,fish,yes,1.0
3,glean,no,0.0
4,shrimp_net,yes,1.0
...,...,...,...
91,shrimp_net,not recorded,NaN
92,shrimp_net,not recorded,NaN
93,shrimp_net,not recorded,NaN
94,glean,not recorded,NaN


In [302]:
df['L4_index']=df['L4_scalar']

### L5
Formal education


In [303]:
print(df['ed_level'].unique())

['primary' 'secondary' 'illiterate' 'NR']


In [304]:
# create a dictionary to replace descriptive scale values with interger scale values
ed_dic = {'illiterate':0,'primary':1,'secondary':2,'NR': None}

# apply function
df['L5_scalar']=df.apply(lambda row: set_scale(ed_dic, row['ed_level']), axis=1 )
df[['pract_cat','ed_level','L5_scalar']]

,pract_cat,ed_level,L5_scalar
0,glean,primary,1.0
1,glean,secondary,2.0
2,fish,illiterate,0.0
3,glean,primary,1.0
4,shrimp_net,secondary,2.0
...,...,...,...
91,shrimp_net,primary,1.0
92,shrimp_net,primary,1.0
93,shrimp_net,illiterate,0.0
94,glean,primary,1.0


In [305]:
df['L5_index']=df.apply(lambda row: index(row['L5_scalar'],0,3), axis=1 )
df[['pract_cat','ed_level','L5_scalar','L5_index']]

,pract_cat,ed_level,L5_scalar,L5_index
0,glean,primary,1.0,0.333333
1,glean,secondary,2.0,0.666667
2,fish,illiterate,0.0,0.000000
3,glean,primary,1.0,0.333333
4,shrimp_net,secondary,2.0,0.666667
...,...,...,...,...
91,shrimp_net,primary,1.0,0.333333
92,shrimp_net,primary,1.0,0.333333
93,shrimp_net,illiterate,0.0,0.000000
94,glean,primary,1.0,0.333333


### L6
Local knowledge

In [306]:
print(df['local_knowledge'].unique())

['agree slightly' 'disagree slightly' 'agree strongly' 'disagree strongly']


In [307]:
# apply function
df['L6_scalar']=df.apply(lambda row: set_scale(likert_dic, row['local_knowledge']), axis=1 )
df[['pract_cat','local_knowledge','L6_scalar']]

,pract_cat,local_knowledge,L6_scalar
0,glean,agree slightly,1
1,glean,disagree slightly,-1
2,fish,agree strongly,2
3,glean,agree strongly,2
4,shrimp_net,agree strongly,2
...,...,...,...
91,shrimp_net,agree strongly,2
92,shrimp_net,agree strongly,2
93,shrimp_net,disagree strongly,-2
94,glean,agree strongly,2


In [308]:
df['L6_index']=df.apply(lambda row: index(row['L6_scalar'],-2,2), axis=1 )
df[['pract_cat','local_knowledge','L6_scalar','L6_index']]

,pract_cat,local_knowledge,L6_scalar,L6_index
0,glean,agree slightly,1,0.75
1,glean,disagree slightly,-1,0.25
2,fish,agree strongly,2,1.00
3,glean,agree strongly,2,1.00
4,shrimp_net,agree strongly,2,1.00
...,...,...,...,...
91,shrimp_net,agree strongly,2,1.00
92,shrimp_net,agree strongly,2,1.00
93,shrimp_net,disagree strongly,-2,0.00
94,glean,agree strongly,2,1.00


## Flexibility
### F1
Capacity to modify fishing practice

In [309]:
print(df['capacity_change_practice'].unique())

['agree strongly' 'disagree strongly' 'agree slightly' 'disagree slightly'
 'neutral']


In [310]:
# apply function
df['F1_scalar']=df.apply(lambda row: set_scale(likert_dic, row['capacity_change_practice']), axis=1 )
df[['pract_cat','capacity_change_practice','F1_scalar']]

,pract_cat,capacity_change_practice,F1_scalar
0,glean,agree strongly,2
1,glean,agree strongly,2
2,fish,agree strongly,2
3,glean,agree strongly,2
4,shrimp_net,agree strongly,2
...,...,...,...
91,shrimp_net,agree strongly,2
92,shrimp_net,agree slightly,1
93,shrimp_net,disagree strongly,-2
94,glean,agree strongly,2


In [311]:
df['F1_index']=df.apply(lambda row: index(row['F1_scalar'],-2,2), axis=1 )
df[['pract_cat','capacity_change_practice','F1_scalar','F1_index']]

,pract_cat,capacity_change_practice,F1_scalar,F1_index
0,glean,agree strongly,2,1.00
1,glean,agree strongly,2,1.00
2,fish,agree strongly,2,1.00
3,glean,agree strongly,2,1.00
4,shrimp_net,agree strongly,2,1.00
...,...,...,...,...
91,shrimp_net,agree strongly,2,1.00
92,shrimp_net,agree slightly,1,0.75
93,shrimp_net,disagree strongly,-2,0.00
94,glean,agree strongly,2,1.00


### F2
Capacity to change livelihood

In [312]:
print(df['capacity_change_livelihood'].unique())

['agree strongly' 'disagree slightly' 'disagree strongly' 'agree slightly']


In [313]:
# apply function
df['F2_scalar']=df.apply(lambda row: set_scale(likert_dic, row['capacity_change_livelihood']), axis=1 )
df[['pract_cat','capacity_change_livelihood','F2_scalar']]

,pract_cat,capacity_change_livelihood,F2_scalar
0,glean,agree strongly,2
1,glean,agree strongly,2
2,fish,agree strongly,2
3,glean,agree strongly,2
4,shrimp_net,agree strongly,2
...,...,...,...
91,shrimp_net,agree strongly,2
92,shrimp_net,agree strongly,2
93,shrimp_net,disagree strongly,-2
94,glean,agree strongly,2


In [314]:
df['F2_index']=df.apply(lambda row: index(row['F2_scalar'],-2,2), axis=1 )
df[['pract_cat','capacity_change_livelihood','F2_scalar','F2_index']]

,pract_cat,capacity_change_livelihood,F2_scalar,F2_index
0,glean,agree strongly,2,1.0
1,glean,agree strongly,2,1.0
2,fish,agree strongly,2,1.0
3,glean,agree strongly,2,1.0
4,shrimp_net,agree strongly,2,1.0
...,...,...,...,...
91,shrimp_net,agree strongly,2,1.0
92,shrimp_net,agree strongly,2,1.0
93,shrimp_net,disagree strongly,-2,0.0
94,glean,agree strongly,2,1.0


## Agency
### AG1
Ability to respond to change

In [315]:
print(df['agency_change_capture'].unique())

["['no_change']" "['leave']" "['fish_less']" 'not sure' "['other']"
 "['change_gear']" "['change_area']"]


In [316]:
def change(response):
    if 'not sure' in response:
        return 0
    elif 'no_change' in response:
        return 0
    else:
        return 1

In [317]:
df['AG1_scalar']=df.apply(lambda row: change(row['agency_change_capture']), axis=1 )
df[['pract_cat','agency_change_capture','AG1_scalar']]

,pract_cat,agency_change_capture,AG1_scalar
0,glean,['no_change'],0
1,glean,['no_change'],0
2,fish,['no_change'],0
3,glean,['no_change'],0
4,shrimp_net,['no_change'],0
...,...,...,...
91,shrimp_net,['no_change'],0
92,shrimp_net,['no_change'],0
93,shrimp_net,['no_change'],0
94,glean,['no_change'],0


In [318]:
df['AG1_index'] =df['AG1_scalar']

### AG2
Decision making power

In [319]:
print(df['decision_making_manag'].unique())

['not involved' 'involved']


In [320]:
# create a dictionary to replace descriptive scale values with interger scale values
dec_dic = {'not involved':0,'involved':1}

# apply function
df['AG2_scalar']=df.apply(lambda row: set_scale(dec_dic, row['decision_making_manag']), axis=1 )
df[['pract_cat','decision_making_manag','AG2_scalar']]

,pract_cat,decision_making_manag,AG2_scalar
0,glean,not involved,0
1,glean,involved,1
2,fish,not involved,0
3,glean,not involved,0
4,shrimp_net,not involved,0
...,...,...,...
91,shrimp_net,not involved,0
92,shrimp_net,involved,1
93,shrimp_net,not involved,0
94,glean,not involved,0


In [321]:
df['AG2_index'] =df['AG2_scalar']

In [322]:
print(df.columns.tolist())

['Unnamed: 0', 'id', 'date', 'ccp', 'gender', 'age', 'birth_place', 'mar_status', 'ed_level', 'house_head', 'glean', 'electricity', 'fridge_freezer', 'fridge_freezer_num', 'water', 'matress', 'matress_num', 'mobile_phone', 'mobile_phone_num', 'smart_phone', 'smart_phone_num', 'fan', 'fan_num', 'tv', 'tv_num', 'radio', 'radio_num', 'stove', 'stove_num', 'transport', 'transport_num', 'floor', 'floor_other', 'wall', 'wall_other', 'roof', 'roof_other', 'access_credit_loans', 'community_participation', 'ccp_member', 'ccp_participation', 'decision_making_manag', 'decision_making_fish', 'change_seagrass', 'direction_change_seagrass', 'causes_change_seagrass', 'change_fish', 'direction_change_resources', 'causes_change_resources', 'link_seagrass_resources', 'capacity_change_practice', 'capacity_change_livelihood', 'capacity_organize', 'local_knowledge', 'agency_change_seagrass', 'agency_change_seagrass_other', 'agency_change_capture', 'agency_change_capture_other', 'action_seagrass', 'action_s

# Combine Indicators


In [323]:

AS_indicators = ['AS1_index', 'AS2_index', 'AS3_index']
O_indicators = ['O1_index', 'O2_index', 'O3_index', 'O4_index']
L_indicators = ['L1_index', 'L2_index',  'L3_index', 'L4_index', 'L5_index','L6_index',]
F_indicators = ['F1_index', 'F2_index']
AG_indicators = ['AG1_index', 'AG2_index']


def calculate_factor(indicators, row):
    val_sum = 0
    indicator_list = []
    for indicator in indicators:
        indicator_val = row[indicator]
        if math.isnan(indicator_val):
            pass
        else:
            val_sum = val_sum + indicator_val
            indicator_list.append(indicator)
    factor = val_sum/len(indicator_list)
    return factor

# apply function
df['Assets']=df.apply(lambda row: calculate_factor(AS_indicators, row), axis=1 )
df['Organization']=df.apply(lambda row: calculate_factor(O_indicators, row), axis=1 )
df['Learning']=df.apply(lambda row: calculate_factor(L_indicators, row), axis=1 )
df['Flexibility']=df.apply(lambda row: calculate_factor(F_indicators, row), axis=1 )
df['Agency']=df.apply(lambda row: calculate_factor(AG_indicators, row), axis=1 )



In [324]:
df['Adaptive Capacity'] = (df['Assets'] + df['Organization']+ df['Learning']+ df['Flexibility']+ df['Agency'])

In [325]:
summary = df.groupby(['pract_cat']).agg(
    Assets=('Assets', np.mean),
    Organization=('Organization', np.mean),
    Flexibility=('Flexibility', np.mean),
    Agency=('Agency', np.mean),
    Adaptive_Capacity=('Adaptive Capacity', np.mean))

summary

,Assets,Organization,Flexibility,Agency,Adaptive_Capacity
pract_cat,,,,,
fish,0.345246,0.871528,0.609375,0.531250,3.047271
glean,0.258334,0.743056,0.791667,0.395833,2.690741
shrimp_net,0.276862,0.680556,0.531250,0.250000,2.297637


In [326]:
summary.to_csv("../adaptive_capacity_matrix_summary.csv")

In [327]:
indicators = AS_indicators + O_indicators + L_indicators + F_indicators + AG_indicators 
dimensions = ['Assets', 'Organization','Learning', 'Flexibility', 'Agency']
cols = indicators + dimensions 
practice = ['pract_cat']
summary_cols = indicators + dimensions + practice
summary= df[summary_cols].groupby(['pract_cat']).agg('mean')


summary.reset_index(inplace=True)

In [330]:
summary = pd.pivot_table(df, values= cols,  columns='pract_cat')
summary

pract_cat,fish,glean,shrimp_net
AG1_index,0.395833,0.500000,0.208333
AG2_index,0.666667,0.291667,0.291667
AS1_index,0.607528,0.521643,0.527511
AS2_index,0.233273,0.045024,0.078525
AS3_index,0.191489,0.208333,0.217391
Agency,0.531250,0.395833,0.250000
Assets,0.345246,0.258334,0.276862
F1_index,0.713542,0.791667,0.604167
F2_index,0.505208,0.791667,0.458333
Flexibility,0.609375,0.791667,0.531250


In [331]:
summary.to_csv("../adaptive_capacity_indicator_matrix_summary.csv")
